In [1]:
import sys
python_path = ['/home/chao/cantera/build/python', '/home/chao/RMG-Py', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages', '/home/chao/cantera', '/home/chao/anaconda3/envs/rmg_env/lib/python37.zip', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/lib-dynload', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/chemprop-0.0.1-py3.7.egg', '/home/chao/anaconda3/envs/rmg_env/lib/python3.7/site-packages/descriptastorus-2.0.0.32-py3.7.egg']
sys.path.clear()
for path in python_path:
    sys.path.append(path)

In [2]:
import cantera as ct
print(ct.__file__)

import yaml
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import rmgpy
import rmgpy.data.base
import rmgpy.molecule
import rmgpy.quantity

/home/chao/cantera/build/python/cantera/__init__.py


In [3]:
path_to_cti = '/home/chao/sketches/cpox_sim/cpox_no_cov_modified/cpox_no_cov_modified.yaml'
gas = ct.Solution(path_to_cti, 'gas')
surf = ct.Interface(path_to_cti, 'surface1', [gas])
species_dict = rmgpy.data.base.Database().get_species('/home/chao/sketches/cpox_sim/cpox_no_cov/species_dictionary.txt',resonance=False)

In [4]:
benergy_c = np.arange(-5.5, -7.75, -0.25)
benergy_o = np.arange(-3.25, -5.5, -0.25)

In [5]:
def find_species_phase_index(species_name):
    """
    Return the phase object (gas or surface) and the index
    of the named species.
    """
    try:
        i = gas.species_index(species_name)
        return gas, i
    except ValueError:
        i = surf.species_index(species_name)
        return surf, i
    
def change_species_enthalpy(species_name, dH, temp):
    """
    Find the species by name and change it's enthlapy by dH (in J/kmol)
    """
    phase, index = find_species_phase_index(species_name)

    species = phase.species(index)
    print(f"Initial H({temp}) = {species.thermo.h(temp)/1e6:.1f} kJ/mol")
    dx = dH / ct.gas_constant  # 'dx' is in fact (delta H / R). Note that R in cantera is 8314.462 J/kmol
    assert isinstance(species.thermo, ct.NasaPoly2)
    # print(species.thermo.coeffs)
    perturbed_coeffs = species.thermo.coeffs.copy()
    perturbed_coeffs[6] += dx
    perturbed_coeffs[13] += dx
    
    species.thermo = ct.NasaPoly2(species.thermo.min_temp, species.thermo.max_temp, 
                            species.thermo.reference_pressure, perturbed_coeffs)
#     print(species.thermo.coeffs)
    phase.modify_species(index, species)
    print(f"Modified H({temp}) = {species.thermo.h(temp)/1e6:.1f} kJ/mol")
    
def correct_binding_energy(target_surf, spec_index, center_atom, x, temperature):
    """
    Here applies linear scaling relationship to calculate binding energy (J/kmol) from one metal
    to another
    Args:
        target_surf: binding energy dictionary of interest eg: Pt_111
        spec_index: index of the species to change
        center_atom: center atom of the species to change
        x: bonds of center atoms of the species to change
    """
    pt_111 = {'H': (-2.75368,'eV/molecule'),
              'C': (-7.02516,'eV/molecule'),
              'N': (-4.63225,'eV/molecule'),
              'O': (-3.81153,'eV/molecule'),
             }
    x_max = {'H': 1,
             'C': 4,
             'N': 3,
             'O': 2,
            }
    deltaE = ((x_max[center_atom]-x) / x_max[center_atom]) * (target_surf[center_atom][0] * 9.6487e7 - pt_111[center_atom][0] * 9.6487e7)
    print(f"Applying LSR correction to {surf.species(spec_index).name}:")
    change_species_enthalpy(surf.species(spec_index).name, deltaE, temperature)

In [6]:
inputs = [('H', 0), ('C', 4), ('C', 2), ('C', 4), ('O', 0), ('C', 2), ('C', 3), ('C', 1), ('C', 0), ('H', 1), ('O', 1),
           ('O', 2), ('C', 3), ('C', 3), ('C', 3), ('C', 1), ('C', 2), ('C', 1), ('C', 1)]

In [7]:
for c_binding_energy in benergy_c:
    for o_binding_energy in benergy_o:
        be_dic = {'C': (c_binding_energy, 'eV/molecule'),
                  'O': (o_binding_energy, 'eV/molecule'),
                  'H': (-2.75368,'eV/molecule')
                 }
        for i in range(1, surf.n_species, 1):
            correct_binding_energy(be_dic, i, inputs[i-1][0], inputs[i-1][1], 700)
        data_root = '/home/chao/sketches/cpox_sim/cpox_no_cov_modified_surfs'
        if not os.path.exists(data_root):
            os.mkdir(data_root)  
#         gas.write_yaml(os.path.join(data_root, f'dm_cpox_c{c_binding_energy}_o{o_binding_energy}.yaml'))
        surf.write_yaml(os.path.join(data_root, f'cpox_nocov_c{c_binding_energy}_o{o_binding_energy}.yaml'))

Applying LSR correction to HX(21):
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol
Applying LSR correction to CO2X(22):
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol
Applying LSR correction to COX(23):
Initial H(700) = -265.1 kJ/mol
Modified H(700) = -191.5 kJ/mol
Applying LSR correction to CH4X(24):
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol
Applying LSR correction to OX(25):
Initial H(700) = -131.5 kJ/mol
Modified H(700) = -77.3 kJ/mol
Applying LSR correction to CH2X(26):
Initial H(700) = 17.1 kJ/mol
Modified H(700) = 90.7 kJ/mol
Applying LSR correction to CH3X(27):
Initial H(700) = -25.4 kJ/mol
Modified H(700) = 11.3 kJ/mol
Applying LSR correction to CHX(28):
Initial H(700) = -13.0 kJ/mol
Modified H(700) = 97.4 kJ/mol
Applying LSR correction to CX(29):
Initial H(700) = 61.4 kJ/mol
Modified H(700) = 208.6 kJ/mol
Applying LSR correction to H2X(30):
Initial H(700) = 7.5 kJ/mol
Modified H(700) = 7.5 kJ/mol
Applying LSR correction to OHX(31

Applying LSR correction to HX(21):
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol
Applying LSR correction to CO2X(22):
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol
Applying LSR correction to COX(23):
Initial H(700) = 520.2 kJ/mol
Modified H(700) = 581.7 kJ/mol
Applying LSR correction to CH4X(24):
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol
Applying LSR correction to OX(25):
Initial H(700) = -428.0 kJ/mol
Modified H(700) = -422.1 kJ/mol
Applying LSR correction to CH2X(26):
Initial H(700) = 802.4 kJ/mol
Modified H(700) = 863.9 kJ/mol
Applying LSR correction to CH3X(27):
Initial H(700) = 367.2 kJ/mol
Modified H(700) = 397.9 kJ/mol
Applying LSR correction to CHX(28):
Initial H(700) = 1164.9 kJ/mol
Modified H(700) = 1257.1 kJ/mol
Applying LSR correction to CX(29):
Initial H(700) = 1631.9 kJ/mol
Modified H(700) = 1755.0 kJ/mol
Applying LSR correction to H2X(30):
Initial H(700) = 7.5 kJ/mol
Modified H(700) = 7.5 kJ/mol
Applying LSR correction t

Applying LSR correction to HX(21):
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol
Applying LSR correction to CO2X(22):
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol
Applying LSR correction to COX(23):
Initial H(700) = 1148.7 kJ/mol
Modified H(700) = 1198.1 kJ/mol
Applying LSR correction to CH4X(24):
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol
Applying LSR correction to OX(25):
Initial H(700) = -821.0 kJ/mol
Modified H(700) = -863.3 kJ/mol
Applying LSR correction to CH2X(26):
Initial H(700) = 1430.8 kJ/mol
Modified H(700) = 1480.3 kJ/mol
Applying LSR correction to CH3X(27):
Initial H(700) = 681.4 kJ/mol
Modified H(700) = 706.1 kJ/mol
Applying LSR correction to CHX(28):
Initial H(700) = 2107.6 kJ/mol
Modified H(700) = 2181.7 kJ/mol
Applying LSR correction to CX(29):
Initial H(700) = 2888.8 kJ/mol
Modified H(700) = 2987.8 kJ/mol
Applying LSR correction to H2X(30):
Initial H(700) = 7.5 kJ/mol
Modified H(700) = 7.5 kJ/mol
Applying LSR correcti

Applying LSR correction to HX(21):
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol
Applying LSR correction to CO2X(22):
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol
Applying LSR correction to COX(23):
Initial H(700) = 1620.3 kJ/mol
Modified H(700) = 1657.7 kJ/mol
Applying LSR correction to CH4X(24):
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol
Applying LSR correction to OX(25):
Initial H(700) = -1310.5 kJ/mol
Modified H(700) = -1401.1 kJ/mol
Applying LSR correction to CH2X(26):
Initial H(700) = 1902.5 kJ/mol
Modified H(700) = 1939.9 kJ/mol
Applying LSR correction to CH3X(27):
Initial H(700) = 917.2 kJ/mol
Modified H(700) = 935.9 kJ/mol
Applying LSR correction to CHX(28):
Initial H(700) = 2815.1 kJ/mol
Modified H(700) = 2871.1 kJ/mol
Applying LSR correction to CX(29):
Initial H(700) = 3832.2 kJ/mol
Modified H(700) = 3907.0 kJ/mol
Applying LSR correction to H2X(30):
Initial H(700) = 7.5 kJ/mol
Modified H(700) = 7.5 kJ/mol
Applying LSR correc

Modified H(700) = 895.6 kJ/mol
Applying LSR correction to CH3OX(61):
Initial H(700) = 829.5 kJ/mol
Modified H(700) = 842.2 kJ/mol
Applying LSR correction to HOCOX(138):
Initial H(700) = 736.8 kJ/mol
Modified H(700) = 749.5 kJ/mol
Applying LSR correction to CHOX(231):
Initial H(700) = 3033.8 kJ/mol
Modified H(700) = 3071.8 kJ/mol
Applying LSR correction to CH2OX(232):
Initial H(700) = 1986.8 kJ/mol
Modified H(700) = 2012.2 kJ/mol
Applying LSR correction to C2H3X(241):
Initial H(700) = 3203.1 kJ/mol
Modified H(700) = 3241.1 kJ/mol
Applying LSR correction to C2HOX(246):
Initial H(700) = 3040.0 kJ/mol
Modified H(700) = 3078.0 kJ/mol
Applying LSR correction to HX(21):
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol
Applying LSR correction to CO2X(22):
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol
Applying LSR correction to COX(23):
Initial H(700) = 1935.2 kJ/mol
Modified H(700) = 1960.5 kJ/mol
Applying LSR correction to CH4X(24):
Initial H(700) = -62.4 kJ/mol
M

Applying LSR correction to HX(21):
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol
Applying LSR correction to CO2X(22):
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol
Applying LSR correction to COX(23):
Initial H(700) = 2080.0 kJ/mol
Modified H(700) = 2081.2 kJ/mol
Applying LSR correction to CH4X(24):
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol
Applying LSR correction to OX(25):
Initial H(700) = -2416.0 kJ/mol
Modified H(700) = -2361.9 kJ/mol
Applying LSR correction to CH2X(26):
Initial H(700) = 2362.2 kJ/mol
Modified H(700) = 2363.4 kJ/mol
Applying LSR correction to CH3X(27):
Initial H(700) = 1147.1 kJ/mol
Modified H(700) = 1147.7 kJ/mol
Applying LSR correction to CHX(28):
Initial H(700) = 3504.6 kJ/mol
Modified H(700) = 3506.4 kJ/mol
Applying LSR correction to CX(29):
Initial H(700) = 4751.5 kJ/mol
Modified H(700) = 4754.0 kJ/mol
Applying LSR correction to H2X(30):
Initial H(700) = 7.5 kJ/mol
Modified H(700) = 7.5 kJ/mol
Applying LSR corr

Applying LSR correction to OX(25):
Initial H(700) = -2742.6 kJ/mol
Modified H(700) = -2712.6 kJ/mol
Applying LSR correction to CH2X(26):
Initial H(700) = 2362.3 kJ/mol
Modified H(700) = 2351.4 kJ/mol
Applying LSR correction to CH3X(27):
Initial H(700) = 1147.1 kJ/mol
Modified H(700) = 1141.7 kJ/mol
Applying LSR correction to CHX(28):
Initial H(700) = 3504.7 kJ/mol
Modified H(700) = 3488.4 kJ/mol
Applying LSR correction to CX(29):
Initial H(700) = 4751.7 kJ/mol
Modified H(700) = 4730.0 kJ/mol
Applying LSR correction to H2X(30):
Initial H(700) = 7.5 kJ/mol
Modified H(700) = 7.5 kJ/mol
Applying LSR correction to OHX(31):
Initial H(700) = -1446.2 kJ/mol
Modified H(700) = -1431.2 kJ/mol
Applying LSR correction to H2OX(32):
Initial H(700) = -247.0 kJ/mol
Modified H(700) = -247.0 kJ/mol
Applying LSR correction to CHOX(33):
Initial H(700) = 968.0 kJ/mol
Modified H(700) = 962.6 kJ/mol
Applying LSR correction to CH3OX(61):
Initial H(700) = 914.6 kJ/mol
Modified H(700) = 909.2 kJ/mol
Applying LSR

Applying LSR correction to HX(21):
Initial H(700) = -20.2 kJ/mol
Modified H(700) = -20.2 kJ/mol
Applying LSR correction to CO2X(22):
Initial H(700) = -405.2 kJ/mol
Modified H(700) = -405.2 kJ/mol
Applying LSR correction to COX(23):
Initial H(700) = 1924.6 kJ/mol
Modified H(700) = 1901.7 kJ/mol
Applying LSR correction to CH4X(24):
Initial H(700) = -62.4 kJ/mol
Modified H(700) = -62.4 kJ/mol
Applying LSR correction to OX(25):
Initial H(700) = -3087.4 kJ/mol
Modified H(700) = -3105.6 kJ/mol
Applying LSR correction to CH2X(26):
Initial H(700) = 2206.8 kJ/mol
Modified H(700) = 2183.8 kJ/mol
Applying LSR correction to CH3X(27):
Initial H(700) = 1069.4 kJ/mol
Modified H(700) = 1057.9 kJ/mol
Applying LSR correction to CHX(28):
Initial H(700) = 3271.4 kJ/mol
Modified H(700) = 3237.1 kJ/mol
Applying LSR correction to CX(29):
Initial H(700) = 4440.7 kJ/mol
Modified H(700) = 4394.9 kJ/mol
Applying LSR correction to H2X(30):
Initial H(700) = 7.5 kJ/mol
Modified H(700) = 7.5 kJ/mol
Applying LSR corr

In [8]:
# def correct_binding_energy(species, delta_atomic_adsoprtion_energies={}):
#     """
#     Changes the thermo of the provided species, by applying a linear scaling relation
#     to correct the adsorption energy.

#     :param species: The species to modify (an RMG Species object)
#     :param delta_atomic_adsoprtion_energies: a dictionary of changes in atomic adsorption energies to apply.
#         mapping for each element an RMG Quantity objects with .value_si giving a value in J/mol.
#     :return: None
#     """
#     molecule = species.molecule[0]
#     # only want/need to do one resonance structure
#     surface_sites = []
#     for atom in molecule.atoms:
#         if atom.is_surface_site():
#             surface_sites.append(atom)
#     normalized_bonds = {'C': 0., 'O': 0., 'N': 0., 'H': 0.}
#     max_bond_order = {'C': 4., 'O': 2., 'N': 3., 'H': 1.}
#     for site in surface_sites:
#         numbonds = len(site.bonds)
#         if numbonds == 0:
#             # vanDerWaals
#             pass
#         else:
#             assert len(site.bonds) == 1, "Each surface site can only be bonded to 1 atom"
#             bonded_atom = list(site.bonds.keys())[0]
#             bond = site.bonds[bonded_atom]
#             if bond.is_single():
#                 bond_order = 1.
#             elif bond.is_double():
#                 bond_order = 2.
#             elif bond.is_triple():
#                 bond_order = 3.
#             elif bond.is_quadruple():
#                 bond_order = 4.
#             else:
#                 raise NotImplementedError("Unsupported bond order {0} for binding energy "
#                                           "correction.".format(bond.order))

#             normalized_bonds[bonded_atom.symbol] += bond_order / max_bond_order[bonded_atom.symbol]


#     # now edit the adsorptionThermo using LSR
#     change_in_binding_energy = 0.0
#     for element in delta_atomic_adsoprtion_energies.keys():
#         change_in_binding_energy += delta_atomic_adsoprtion_energies[element].value_si *  normalized_bonds[element]
#     print(molecule, change_in_binding_energy, normalized_bonds)
#     if change_in_binding_energy != 0.0:
#         print(f"Applying LSR correction to {species.label}:")
#         change_species_enthalpy(species.label, change_in_binding_energy*1000, 300)

In [9]:
# pt_111 = {'H': -2.75368,'C': -7.02516,'N': -4.63225,'O': -3.81153}
# for i in energy_c:
#     for j in energy_o:
#         delta_atomic_adsoprtion_energies = {}
#         delta_atomic_adsoprtion_energies['C'] = rmgpy.quantity.Energy(i -pt_111['C'], 'eV/molecule')
#         delta_atomic_adsoprtion_energies['O'] = rmgpy.quantity.Energy(j - pt_111['O'], 'eV/molecule')
#         for species in surf.species():
#             rmg_spcs = species_dict[species.name]
#             correct_binding_energy(rmg_spcs, delta_atomic_adsoprtion_energies)
#         surf.write_yaml( f'cpox_nocov_c{i}_o{j}.yaml')
        

In [10]:
energy_c = np.arange(-5.5, -7.75, -0.25)
energy_o = np.arange(-3.25, -5.5, -0.25)
with open('/home/chao/sketches/cpox_sim/cpox_no_cov_modified/cpox_no_cov_modified.yaml', 'r') as f:
    model = yaml.load(f, Loader=yaml.FullLoader)
for i in energy_c:
    for j in energy_o:
        path_to_cti = f'/home/chao/sketches/cpox_sim/cpox_no_cov_modified_surfs/cpox_nocov_c{i}_o{j}.yaml'
        with open(path_to_cti, 'r') as f:
            model_to_change = yaml.load(f, Loader=yaml.FullLoader)

        model_to_change['elements'] = model['elements']

        with open(path_to_cti, 'w') as f:
            yaml.dump(model_to_change, f, default_flow_style=False)
            
        gas = ct.Solution(path_to_cti, 'gas')
        surf = ct.Interface(path_to_cti, 'surface1', [gas])